In [ ]:
import numpy as np
import argparse
import imutils
import time
import cv2
import os
from imutils.video import VideoStream
from imutils.video import FPS
from matplotlib import pyplot as plt

In [ ]:
# load the COCO class labels our YOLO model was trained on
LABELS = open('yolo-coco/coco.names').read().strip().split("\n")

In [ ]:
# initialize a list of colors to represent each possible class label
# np.random.seed(42)
COLOURS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

In [ ]:
net = cv2.dnn.readNetFromDarknet('yolo-coco/yolov3.cfg', 'yolo-coco/yolov3.weights')
outputLayers = [net.getLayerNames()[i[0] - 1] for i in net.getUnconnectedOutLayers()]



In [ ]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
fps = FPS().start()

In [ ]:
while True:
    frame = vs.read()

    frame = imutils.resize(frame, width=400)

    (H, W) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)

    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(outputLayers)
    end = time.time()

    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes, confidences, classIDs = [], [], []

    # loop over each of the 3 layer output layers
    for output in layerOutputs:
		# loop over each detection for each output layer
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

           
            if confidence > 0.5:
           
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
                # apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

	# ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLOURS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    
    
   # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    # update the FPS counter
    fps.update()
    
    
# stop the timer and display FPS information
fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


In [ ]:
layerOutputs[0][0][5:].shape

In [ ]:
for blah in layerOutputs:
    print(blah.shape)


In [ ]:
imgL = cv2.imread('images/leftStereo.jpg',0)
imgR = cv2.imread('images/rightStereo.jpg',0)


plt.imshow(imgL)
plt.show()
plt.imshow(imgR)
plt.show()

In [ ]:
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()
